In [2]:
from mesa.batchrunner import BatchRunner, BatchRunnerMP
from mesa.time import RandomActivation
from model import *
from customizeTime import *
import pickle
import numpy as np
import uuid

from customizeTime import *
import env_settings

TYPE_MAP = {0:'vert',
            1:'horz'}

settings.MUTATION_PRO = 0.01

class A_5(GenAgent):
    def muta_genetic_info(self):
        # no need for mutation
        pass
    
        
class M_5(GenModel):
    def get_agent_config(self):
        return A_5
    
    def init_env(self):
        #no influence
        return list(np.random.random(10000))
    
    def get_schedule(self):
        return RandomActivationMutationWithConstrain
    #
    def __init__(self, N, width, height, specific_kargs):
        #mut_var, init_var, env_val
        self.model_name = 'M5'
        
        init_ratio = 0.5
        self.env_val = 0
        
        settings.LIFE_TIME = specific_kargs['lifetime']
         
        settings.INIT_MEAN = 0.5
        settings.INIT_SD = specific_kargs['gene_sd']
        settings.INIT_LOW = settings.INIT_MEAN - 1 * 2 * settings.INIT_SD
        settings.INIT_HIGH = settings.INIT_MEAN + 1 * 2 * settings.INIT_SD

        settings.POPU_MUTA_SD = specific_kargs['muta_sd']
        settings.POPU_MUTA_LOW = -2 * settings.POPU_MUTA_SD
        settings.POPU_MUTA_HIGH = 2 * settings.POPU_MUTA_SD
        
        self.specific_kargs_str = '_'.join([k + "_" + str(specific_kargs[k]) for k in specific_kargs])
        super().__init__(N, width, height, init_ratio)
        

    
    def model_level_record_init(self):
        data_file_name = check_path("./res/")
        data_file_name = check_path(data_file_name + self.model_name)
        data_file_name = check_path(data_file_name + "/" + self.specific_kargs_str)
        data_file_name = data_file_name + '/' + str(uuid.uuid1())
        print('data in file ', data_file_name)
        self.data_file = open(data_file_name, 'w')
        self.data_file.write('#' + '\t'.join(
            ('step',
             TYPE_MAP[0], TYPE_MAP[0] + '_beta', TYPE_MAP[0] + '_gene', 'var',  
             TYPE_MAP[1], TYPE_MAP[1] + '_beta', TYPE_MAP[1] + '_gene', 'var'))+ '\n')
       
    def get_popu_beta(self, gen_type):
        return np.mean([agent.self_gen_cof for agent in self.schedule.agents if agent.gen_type == gen_type])
    
    def get_gene_mean(self, gen_type):
        return [np.mean(agent.gen_info) for agent in self.schedule.agents if agent.gen_type == gen_type]
    
    def get_popu_gene_mean(self, gene_mean):
        return np.mean(gene_mean)

    def get_popu_gene_var(self, gene_mean):
        return np.var(gene_mean)

    def collect_data(self):
        ver_mean = self.get_gene_mean(0)
        hor_mean = self.get_gene_mean(1)
        return (self.schedule.steps,
                self.get_ver_popu_size(), self.get_popu_beta(0), self.get_popu_gene_mean(ver_mean), self.get_popu_gene_var(ver_mean),
                self.get_her_popu_size(), self.get_popu_beta(1), self.get_popu_gene_mean(hor_mean), self.get_popu_gene_var(hor_mean))
        
        
    def step(self):
        
        data = self.collect_data()
        #print(data)
        self.data_file.write('\t'.join([str(d) for d in data]) + '\n')
        if self.schedule.steps % 100 == 0: 
            self.data_file.flush()
        super().step()
        

In [3]:
import sys

#0.01, 0.005, 0.001
sys.argv = 'm5 0.03 3'.split(' ')
#mutation_sd, lifetime

muta_sd, lifetime = float(sys.argv[1]), float(sys.argv[2])

fixed_params = {'N':2000, 'width':100, 'height':100}

variable_params = {'specific_kargs': [{'lifetime': lifetime, 'gene_sd':muta_sd, 'muta_sd':muta_sd}]}

batch_run = BatchRunnerMP(
    M_5, 1,
    variable_parameters=variable_params,
    fixed_parameters=fixed_params,
    iterations=1,
    max_steps=5000,
)
batch_run.run_all()

0it [00:00, ?it/s]

data in file  ./res/M5/lifetime_3.0_gene_sd_0.03_muta_sd_0.03/4f19fae0-d6cc-11e9-bcf6-f2189850a77c
0 population size:  2000
1 population size:  2759
2 population size:  4141
3 population size:  4929


Process ForkPoolWorker-1:



KeyboardInterrupt: 

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/site-packages/multiprocess/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.7/site-packages/pathos/helpers/mp_helper.py", line 15, in <lambda>
    func = lambda args: f(*args)
  File "/usr/local/lib/python3.7/site-packages/mesa/batchrunner.py", line 153, in run_iteration
    self.run_model(model)
  File "/usr/local/lib/python3.7/site-packages/mesa/batchrunner.py", line 178, in run_model
    model.step()
  File "<ipython-input-2-ee2e59721451>", line 96, in step
    super().step()
  File "/Users/zhuliu/projects/ye/selection/model.py", line 151, in step
    self.schedule.step()
  File "/Users/zhuliu/projects/ye/selection/custom